# Word Relatedness

This notebook calculate the word relatedness using gensim and word2vec dataset.

In [55]:
from IPython.display import HTML
from PIL import Image
from io import BytesIO

import base64
import ctypes
import numpy as np
import pandas as pd
import PIL
import scipy.io as sio

In [ ]:
import gensim
import re
import pandas

# This should be a separate snippet because it take a long time to load
model = gensim.models.KeyedVectors.load_word2vec_format('/Volumes/Spaceship/Alon_Gist/gensim/model/GoogleNews-vectors-negative300.bin.gz', binary=True)



In [ ]:
gist_word_data = pd.read_csv('actual_word_table.csv', usecols=["image", "soa", "word", "frequency"])
gist_word_data.head()


In [ ]:
alon_shinji_data = pd.read_csv('alon_image_all_descriptors.csv', encoding="ISO-8859-1")
alon_shinji_data.head()

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools

image_files = []
for row in gist_word_data.iterrows():
    image_file = row[1]['image']
    image_files.append(image_file)

unique_files = list(set(image_files))
for img in unique_files:
    print(img + "....")
    low_words = gist_word_data.loc[(gist_word_data['image'] == img) & (gist_word_data['soa'] == 67)]
    medium_words = gist_word_data.loc[(gist_word_data['image'] == img) & (gist_word_data['soa'] == 133)]
    high_words = gist_word_data.loc[(gist_word_data['image'] == img) & (gist_word_data['soa'] == 267)]

    low_words = ",".join(list(low_words['word']))
    medium_words = ",".join(list(medium_words['word']))
    high_words = ",".join(list(high_words['word']))
    
    image = image_file.replace(".jpg", "").replace("im", "").lstrip("0")

    unlimited_words = alon_shinji_data.loc[alon_shinji_data['imageNum'] == int(image)]
    unlimited_words = unlimited_words.iloc[0]['descriptors']
    
    documents = [str(low_words), 
                 str(medium_words), 
                 str(high_words), 
                 str(unlimited_words)]
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    all_words = []
    all_words.append(low_words)
    all_words.append(medium_words)
    all_words.append(high_words)
    
    word_list = str(all_words).split(',')
    word_list = list(set(word_list))
    pairwise_list = list(itertools.combinations(word_list, 2))
    similarity_scores = [[0 for x in range(len(word_list))] for y in range(len(word_list))] 

    for x in range(len(word_list)):
        for y in range(len(word_list)):
            if x == y:
                similarity_scores[x][y] = 1
                
    for pair in pairwise_list:
        try:
            idx1 = word_list.index(pair[0])
            idx2 = word_list.index(pair[1])
            similarity_scores[idx1][idx2] = model.similarity(pair[0], pair[1])
        except KeyError as err:    
            error_message = "{0}".format(err);
            similarity_scores.append(-1)

    for i in range(0, len(pairwise_list)):
        pw = pairwise_list[i]
        
        idx1 = word_list.index(pw[0])
        idx2 = word_list.index(pw[1])
        
        #print("%s vs %s: %0.4f" % (pw[0], pw[1], similarity_scores[idx1][idx2]))
    
    from scipy.cluster.hierarchy import dendrogram, linkage
    
    Z = linkage(similarity_scores, 'ward')

    %matplotlib inline
    # calculate full dendrogram
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('sample index')
    plt.ylabel('distance')
    dendrogram(
        Z,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=20.,  # font size for the x axis labels
        leaf_label_func=lambda v: word_list[v],
    )
    
    plt.show()    

#     mat=np.random.random((4,4))
#     mat[0,] = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
#     mat[1,] = cosine_similarity(tfidf_matrix[1:2], tfidf_matrix)
#     mat[2,] = cosine_similarity(tfidf_matrix[2:3], tfidf_matrix)
#     mat[3,] = cosine_similarity(tfidf_matrix[3:4], tfidf_matrix)

#     print(mat)



#     plt.imshow(mat);
#     plt.colorbar()
#     plt.show()


im0004473.jpg....


/Users/Zhao/.pyenv/versions/3.5.0/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ValueError: setting an array element with a sequence.